# Requirement
1. Sử dụng ngôn ngữ lập trình Python và thư viện sklearn, viết các hàm để thực hiện các mô hình hồi quy tuyến tính sau:
$$
\begin{aligned}
eq1 = \underset{\beta}{\operatorname{argmin}}\|\mathbf{y} - \mathbf{X}\beta\|_2^2 + \rho\|\beta\|_1 \\
eq2 = \underset{\beta}{\operatorname{argmin}}\|\mathbf{y} - \mathbf{X}\beta\|_2^2 + \rho\|\beta\|_2^2
\end{aligned}
$$

Khi nhận đầu vào bao gồm $\mathbf{X} \in \mathbb{R}^{n \times p}$, $\mathbf{y} \in \mathbb{R}^n$, và hằng số $\rho \in \mathbb{R}$, các hàm này trả về lời giải tối ưu $\beta$ khi giải quyết loại bài toán hồi quy tuyến tính cụ thể (Lasso hoặc Ridge).      

2. Sử dụng các hàm ở trên, huấn luyện các mô hình hồi quy tuyến tính được chính quy hóa dạng $l_1$ hoặc $l_2$ trên các tập dữ liệu `data_a.csv` và `data_b.csv`. Cột cuối cùng trong mỗi tập dữ liệu là biến mục tiêu $\mathbf{y}$, và các cột còn lại tạo nên ma trận các biến độc lập $\mathbf{X}$. Với mỗi tập dữ liệu, chia thành $50\%$ cho huấn luyện, $25\%$ kiểm thử, và $25\%$ cho kiểm tra. Sử dụng tập kiểm thử để chọn giá trị tối ưu cho các tham số $\rho$ trong dãy $[0,001, 0.01, 0.1, 1, 2]$ cho các mô hình. Tính $\|\mathbf{y} - \mathbf{X}\beta\|_2$ trên tập kiểm tra sử dụng giá trị $\beta$ từ các mô hình hồi quy tuyến tính được chính quy hóa dạng $l_1$, $l_2$ và dạng thông thường ($\rho = 0$). Bằng việc so sánh thêm với mô hình đơn giản nhất là dự đoán sử dụng giá trị trung bình $\mathbf{y}$ của tập huấn luyện và kiểm thử, cho biết phương pháp nào cho kết quả tốt nhất?        

3. Dựa vào các kết quả trên hai tập dữ liệu, lợi ích của các mô hình hồi quy tuyến tính được chính quy hóa là gì? Trường hợp nào chúng ta nên sử dụng phương pháp hồi quy tuyến tính được chính quy hóa so với dạng thông thường?

In [8]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

# Give a header to csv file

In [3]:
# df = pd.read_csv('data_a.csv')
# column_name = ['feature_' + str(i) for i in range(len(df.columns))]
# df.to_csv("data_a_preprocessed.csv", header=column_name, index=False)

In [5]:
# df = pd.read_csv('data_b.csv')
# column_name = ['feature_' + str(i) for i in range(len(df.columns))]
# df.to_csv("data_b_preprocessed.csv", header=column_name, index=False)

# Read preprocessed file

In [6]:
df_a = pd.read_csv('data_a_preprocessed.csv')
df_a

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122
0,0.09,0.43,0.51,0.58,0.04,0.01,0.58,0.56,0.50,0.52,...,0.14,0.11,0.19,0.05,0.01,0.75,0.0,0.60,0.10,0.31
1,0.05,0.59,0.23,0.39,0.09,1.00,0.45,0.55,0.37,0.45,...,0.01,0.73,0.28,0.00,0.02,0.64,0.0,1.00,0.23,0.50
2,0.08,0.39,0.08,0.85,0.04,0.25,0.33,0.48,0.27,0.47,...,0.07,0.21,0.04,0.02,0.01,0.70,1.0,0.44,0.11,0.14
3,0.29,0.42,0.27,0.64,0.18,0.31,0.33,0.45,0.28,0.55,...,0.05,0.87,1.00,0.21,0.14,0.69,0.5,0.61,0.34,0.30
4,0.07,0.34,0.08,0.84,0.32,0.05,0.28,0.53,0.32,0.49,...,0.01,0.89,1.00,0.00,0.01,0.88,0.0,0.35,0.09,0.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,0.04,0.39,0.39,0.65,0.09,0.13,0.30,0.40,0.25,0.67,...,0.03,0.28,0.32,0.02,0.01,0.85,0.0,0.99,0.19,0.22
117,0.03,0.32,0.61,0.47,0.09,0.07,0.32,0.49,0.31,0.50,...,0.01,0.47,0.42,0.07,0.08,0.49,0.0,0.37,1.00,0.45
118,0.07,0.38,0.17,0.84,0.11,0.04,0.35,0.41,0.30,0.64,...,0.09,0.13,0.17,0.02,0.01,0.72,0.0,0.62,0.15,0.07
119,0.16,0.37,0.25,0.69,0.04,0.25,0.35,0.50,0.31,0.54,...,0.08,0.32,0.18,0.08,0.06,0.78,0.0,0.91,0.28,0.23


In [7]:
df_b = pd.read_csv('data_b_preprocessed.csv')
df_b

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13
0,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
1,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
2,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
3,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
4,0.02985,0.0,2.18,0,0.458,6.430,58.7,6.0622,3,222,18.7,394.12,5.21,28.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
501,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
502,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
503,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0
